In [2]:
import dask.dataframe as dd
import pandas as pd
import os
import itertools
import numpy as np

In [3]:
#import network files
os.chdir('/scratch/bacint/jdeleuw/ananse/network_data')
Day0 = dd.read_csv(os.getcwd() + '/Day0_network.txt', sep='\t|_', engine='python')
Day07 = dd.read_csv(os.getcwd() + '/Day07_network.txt', sep='\t|_', engine='python')
Day14 = dd.read_csv(os.getcwd() + '/Day14_network.txt', sep='\t|_', engine='python')
Day21 = dd.read_csv(os.getcwd() + '/Day21_network.txt', sep='\t|_', engine='python')
Day43 = dd.read_csv(os.getcwd() + '/Day43_network.txt', sep='\t|_', engine='python')
Day60 = dd.read_csv(os.getcwd() + '/Day60_network.txt', sep='\t|_', engine='python')
KC = dd.read_csv(os.getcwd() + '/KC_network.txt', sep='\t|_', engine='python')

In [4]:
TFs = pd.read_csv('/scratch/bacint/jdeleuw/ananse/analysis/interesting_TFs.csv', header = None)

In [5]:
Cluster1 = ['FOS', 'MYC']
Cluster2 = ['ZSCAN10', 'ZIC3', 'SOX2', 'PRDM14', 'SALL4', 'OTX2', 'NANOG', 'FOXH1', 'MYBL2']
Cluster3 = ['OSR1', 'MSX1']
Cluster4 = ['HAND1', 'MEIS1', 'HOXB2', 'MEIS2', 'MSX2', 'TBX3', 'HEY1', 'TFAP2B']
Cluster5 = ['TP63', 'IRX3', 'NFIC', 'HES2', 'EHF']
Cluster6 = ['TFAP2A', 'GRHL1', 'IRF6', 'KLF5', 'SNAI2', 'EPAS1', 'CEBPD', 'FOSL2']

In [6]:
#filter the TFs of of all the clusters from the network files
TFs = Cluster1 + Cluster2 + Cluster3 + Cluster4 + Cluster5 + Cluster6
ddf_networks = [Day0, Day07, Day14, Day21, Day43, Day60, KC]
output_names = ['filtered_' + i for i in dir() if i.startswith('Day')] + ['filtered_KC']

for ddf_network, output_name in zip(ddf_networks, output_names): 
    filter1 = ddf_network.source.isin(TFs)
    filter2 = ddf_network.target.isin(TFs)
    filtered_network = ddf_network[filter1 & filter2]
    globals()[output_name] = filtered_network.compute().reset_index(drop = True)

In [7]:
filtered_Day07

,source,target,binding,prob
0,CEBPD,HES2,0.693095,0.876688
1,EHF,HES2,0.412528,0.360474
2,EPAS1,HES2,0.731012,0.916092
3,FOS,HES2,0.709216,0.894671
4,FOSL2,HES2,0.621529,0.774385
...,...,...,...,...
1117,TFAP2A,FOXH1,0.655477,0.827578
1118,TFAP2B,FOXH1,0.680688,0.861617
1119,TP63,FOXH1,0.530482,0.602789
1120,ZIC3,FOXH1,0.413487,0.362352


In [11]:
#Make table of average interaction score in each comparison
network_dfs = [globals()[i] for i in dir() if i.startswith('filtered_Day')] + [filtered_KC]

Days = ['D0', 'D7', 'D14', 'D21', 'D43', 'D60', 'KC']
interaction_average = pd.DataFrame(index = TFs, columns = Days)

hm_data = pd.DataFrame(index = TFs, columns = Days)

for df, day in zip(network_dfs, Days):  
    interactions = pd.DataFrame(index = TFs, columns = TFs)
    for TF, target in itertools.product(TFs, repeat = 2):
        filtered_target = df[df['target'].isin([target])]
        filtered_TF = filtered_target[filtered_target['source'].isin([TF])]

        interactions.at[target, TF] = filtered_TF['prob'].sum()
    for TF in TFs:
        interactions.replace(0, np.nan, inplace=True)    
        hm_data.at[TF, day] = interactions.loc[:, TF].mean()

hm_data.to_csv('/scratch/bacint/jdeleuw/ananse/analysis/avg_interactions.tsv', sep = '\t')  

In [12]:
hm_data

,D0,D7,D14,D21,D43,D60,KC
FOS,0.853078,0.843789,0.791552,0.877669,0.869291,0.885185,0.762134
MYC,0.834050,0.665347,0.627863,0.796841,0.830851,0.873278,0.829791
ZSCAN10,0.829359,0.536939,0.501147,0.506793,0.442580,0.408675,0.380859
ZIC3,0.832309,0.488255,0.516427,0.565373,0.551847,0.485894,0.384778
SOX2,0.841688,0.602688,0.624394,0.567919,0.562535,0.574997,0.380236
PRDM14,0.842331,0.527082,0.470077,0.490462,0.445596,0.527684,0.487014
SALL4,0.840303,0.806001,0.637401,0.594907,0.565082,0.590737,0.499527
OTX2,0.826925,0.486919,0.429252,0.575183,0.510884,0.509749,0.373711
NANOG,0.840365,0.649032,0.683979,0.687427,0.627291,0.597069,0.567702
FOXH1,0.821157,0.550599,0.475506,0.457730,0.449579,0.594801,0.479235
